In [1]:
# Import the libraries
import os
import cv2
import csv
import numpy as np

In [2]:
# CoverDescriptor (Extract the Keypoints and the Descriptor from an image)
class CoverDescriptor:
    def __inti__(self, useSIFT=False):
        self.useSIFT = useSIFT
        
    def describe(self, image):
        # BRISK Descriptor
        descriptor = cv2.BRISK_create()
        # If True using SIFT Descriptor
        #if self.useSIFT:
            #descriptor = cv2.xfeatures2d.SIFT_create()
        
        # Computing the keypoints and the descriptor for an image    
        (kps, descs) = descriptor.detectAndCompute(image, None)
        # Extracting only the points using .pt in the Keypoints
        kps = np.float32([kp.pt for kp in kps])
        
        return (kps, descs)

In [3]:
# CoverMatcher ()
class CoverMatcher:
    def __init__(self, descriptor, coverPaths, ratio=0.7, minMatches=40, useHamming=True):
        self.descriptor = descriptor
        self.coverPaths = coverPaths
        self.ratio = ratio
        self.minMatches  = minMatches
        self.distanceMethod = "BruteForce"
        
        if useHamming:
            self.distanceMethod += "-Hamming"
            
    def search(self, queryKps, queryDescs):
        results = {}
        
        for coverPath in coverPaths:
            image = cv2.imread(coverPath)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            (kps, descs) = self.descriptor.describe(gray)
            
            score = self.match(queryKps, queryDescs, kps, descs)
            results[coverPath] = score
            
            if len(results) > 0:
                results = sorted([(v, k) for (k, v) in results.items() if v>0], reverse=True)
                return results
     
    def match(self, kpsA, featuresA, kpsB, featuresB):
        matcher = cv2.DescriptorMatcher_create(self.distanceMethod)
        rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
        
        matches = []
        
        for m in rawMatches:
            if len(m) == 2 and m[0].distance > m[1].distance * self.ratio:
                matches.append((m[0].trainIdx, m[0].queryIdx))
                
        if len(matches) > self.minMatches:
            ptsA = np.float32([kpsA[i] for (i, _) in matches])
            ptsB = np.float32([kpsA[j] for (_, j) in matches])
            
            (_, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, 4.0)
            
            return float(status.sum()) / status.size
        
        return -1.0

In [4]:
db = {}

for l in csv.reader(open("data/books.csv")):
    db[l[-1]] = l[1:3]

In [5]:
coverPaths = []

for path in os.listdir("data/covers"):
    coverPaths.append("data/covers/"+path)
coverPaths[0]    

'data/covers/0000001.jpg'

In [7]:
useSIFT = False
useHamming = True
ratio = 0.7
minMatches = 40

if useSIFT:
    minMatches=50

In [ ]:
cd = CoverDescriptor()
cv = CoverMatcher(cd, coverPaths, ratio=ratio, minMatches=minMatches, useHamming=useHamming)

queryImage = cv2.imread("data/query/query3.jpg")
gray = cv2.cvtColor(queryImage, cv2.COLOR_BGR2GRAY)
(queryKps, queryDescs) = cd.describe(gray)

results = cv.search(queryKps, queryDescs)
cv2.imshow("Query", queryImage)

if len (results) == 0:
    print("I could not find a match for that cover!" )
    cv2.waitKey(0)
else:
    for (i, (score, coverPath)) in enumerate(results):
        (author, title) = db[coverPath]
        print("{}. {:.2f}% : {} - {}" .format(i + 1, score * 100, author, title))
    
        result = cv2.imread(coverPath)
        cv2.imshow("Result", result)
        cv2.waitKey(0)

1. 3.27% : Robert C. Martin - Clean Code
